# अध्याय 7 परिणाम जांचें

- [आई. पर्यावरण विन्यास](#आई. पर्यावरण विन्यास)
- [II. जांचें कि क्या आउटपुट में संभावित रूप से हानिकारक सामग्री है](#II. जांचें कि क्या आउटपुट में संभावित रूप से हानिकारक सामग्री है)
- [III. जांचें कि क्या आउटपुट परिणाम प्रदान की गई उत्पाद जानकारी के अनुरूप हैं](#III. जांचें कि क्या आउटपुट परिणाम प्रदान की गई उत्पाद जानकारी के अनुरूप हैं)

इस अध्याय में, हम इस बात पर ध्यान केंद्रित करेंगे कि सिस्टम द्वारा उत्पन्न आउटपुट की समीक्षा कैसे करें, उपयोगकर्ताओं को दिखाने से पहले आउटपुट की गुणवत्ता, प्रासंगिकता और सुरक्षा की समीक्षा करना, प्रदान की गई प्रतिक्रिया सुनिश्चित करने के लिए महत्वपूर्ण है, चाहे स्वचालित प्रक्रिया में या अन्य परिदृश्यों में। हम सीखेंगे कि आउटपुट का मूल्यांकन करने के लिए समीक्षा एपीआई का उपयोग कैसे करें और आउटपुट दिखाने से पहले मॉडल के गुणवत्ता मूल्यांकन में सुधार के लिए अतिरिक्त संकेतों का उपयोग कैसे करें।

## 1. पर्यावरण विन्यास

पिछले अध्याय की तरह, हमें सबसे पहले OpenAI API का उपयोग करने के लिए पर्यावरण को कॉन्फ़िगर करना होगा

In [1]:
import openai
# तृतीय-पक्ष लाइब्रेरी आयात करें

openai.api_key  = "sk-..."
# API_KEY सेट करें, कृपया इसे अपनी API_KEY से बदलें

# निम्नलिखित पर्यावरण चर पर आधारित कॉन्फ़िगरेशन विधि का एक उदाहरण है, जो अधिक सुरक्षित है और इसे बाद में कवर नहीं किया जाएगा।
#आयात openai
# ओएस आयात करें
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    '''
    封装一个访问 OpenAI GPT3.5 的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message["content"]

## 2. जांचें कि क्या आउटपुट में संभावित रूप से हानिकारक सामग्री है
मुख्य रूप से मॉडरेशन एपीआई का उपयोग

In [ ]:
final_response_to_customer = f"""
The SmartX ProPhone has a 6.1-inch display, 128GB storage, \
12MP dual camera, and 5G. The FotoSnap DSLR Camera \
has a 24.2MP sensor, 1080p video, 3-inch LCD, and \
interchangeable lenses. We have a variety of TVs, including \
the CineView 4K TV with a 55-inch display, 4K resolution, \
HDR, and smart TV features. We also have the SoundMax \
Home Theater system with 5.1 channel, 1000W output, wireless \
subwoofer, and Bluetooth. Do you have any specific questions \
about these products or any other products we offer?
"""
# मॉडरेशन OpenAI का कंटेंट समीक्षा फ़ंक्शन है, जिसका उपयोग इस सामग्री की हानिकारक सामग्री का पता लगाने के लिए किया जाता है

response = openai.Moderation.create(
    input=final_response_to_customer
)
moderation_output = response["results"][0]
print(moderation_output)

{
  "categories": {
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": false,
    "violence/graphic": false
  },
  "category_scores": {
    "hate": 2.6680607e-06,
    "hate/threatening": 1.2194433e-08,
    "self-harm": 8.294434e-07,
    "sexual": 3.41087e-05,
    "sexual/minors": 1.5462567e-07,
    "violence": 6.3285606e-06,
    "violence/graphic": 2.9102332e-06
  },
  "flagged": false
}


In [4]:
final_response_to_customer = f"""
SmartX ProPhone 有一个 6.1 英寸的显示屏，128GB 存储、\
1200 万像素的双摄像头，以及 5G。FotoSnap 单反相机\
有一个 2420 万像素的传感器，1080p 视频，3 英寸 LCD 和\
可更换的镜头。我们有各种电视，包括 CineView 4K 电视，\
55 英寸显示屏，4K 分辨率、HDR，以及智能电视功能。\
我们也有 SoundMax 家庭影院系统，具有 5.1 声道，\
1000W 输出，无线重低音扬声器和蓝牙。关于这些产品或\
我们提供的任何其他产品您是否有任何具体问题？
"""

response = openai.Moderation.create(
    input=final_response_to_customer
)
moderation_output = response["results"][0]
print(moderation_output)

{
  "categories": {
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": false,
    "violence/graphic": false
  },
  "category_scores": {
    "hate": 2.6680607e-06,
    "hate/threatening": 1.2194433e-08,
    "self-harm": 8.294434e-07,
    "sexual": 3.41087e-05,
    "sexual/minors": 1.5462567e-07,
    "violence": 6.3285606e-06,
    "violence/graphic": 2.9102332e-06
  },
  "flagged": false
}


जैसा कि आप देख सकते हैं, इस आउटपुट को लेबल नहीं किया गया है और सभी श्रेणियों में बहुत कम अंक प्राप्त होते हैं, जो दर्शाता है कि दी गई प्रतिक्रिया उचित है।

सामान्य तौर पर, आउटपुट की समीक्षा करना भी बहुत महत्वपूर्ण है। उदाहरण के लिए, यदि आप संवेदनशील दर्शकों के लिए चैटबॉट बना रहे हैं, तो आप आउटपुट को लेबल करने के लिए कम सीमा का उपयोग कर सकते हैं, यदि आउटपुट की समीक्षा करना इंगित करता है कि सामग्री क्या है ध्वजांकित, आप उचित कार्रवाई कर सकते हैं, जैसे वैकल्पिक उत्तर के साथ जवाब देना या नया उत्तर तैयार करनाप्रायोजन.

ध्यान दें कि जैसे-जैसे हम अपने मॉडलों में सुधार करते हैं, उनमें कोई हानिकारक आउटपुट देने की संभावना भी कम होती जाती है।

आउटपुट की समीक्षा करने का दूसरा तरीका यह है कि मॉडल से ही पूछें कि क्या उसके द्वारा उत्पन्न परिणाम संतोषजनक हैं और आपके द्वारा परिभाषित मानकों को पूरा करते हैं। यह मॉडल में इनपुट के हिस्से के रूप में उत्पन्न आउटपुट को फीड करके और गुणवत्ता का मूल्यांकन करने के लिए कहकर किया जा सकता है आउटपुट का। आप इसे विभिन्न तरीकों से कर सकते हैं। आइए एक उदाहरण देखें।

## 3. जांचें कि क्या आउटपुट परिणाम प्रदान की गई उत्पाद जानकारी के अनुरूप हैं

In [7]:
# यह इलेक्ट्रॉनिक उत्पादों से संबंधित जानकारी है
system_message = f"""
You are an assistant that evaluates whether \
customer service agent responses sufficiently \
answer customer questions, and also validates that \
all the facts the assistant cites from the product \
information are correct.
The product information and user and customer \
service agent messages will be delimited by \
3 backticks, i.e. ```.
Respond with a Y or N character, with no punctuation:
Y - if the output sufficiently answers the question \
AND the response correctly uses product information
N - otherwise

Output a single letter only.
"""

#यह एक ग्राहक का प्रश्न है
customer_message = f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one. \
Also tell me about your tvs"""
product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""

q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question?

Output Y or N
"""
# प्रासंगिकता निर्धारित करें
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

Y


In [ ]:
# यह इलेक्ट्रॉनिक उत्पादों से संबंधित जानकारी है
system_message = f"""
您是一个助理，用于评估客服代理的回复是否充分回答了客户问题，\
并验证助理从产品信息中引用的所有事实是否正确。 
产品信息、用户和客服代理的信息将使用三个反引号（即 ```)\
进行分隔。 
请以 Y 或 N 的字符形式进行回复，不要包含标点符号：\
Y - 如果输出充分回答了问题并且回复正确地使用了产品信息\
N - 其他情况。

仅输出单个字母。
"""

#यह एक ग्राहक का प्रश्न है
customer_message = f"""
告诉我有关 smartx pro 手机\
和 fotosnap 相机（单反相机）的信息。\
还有您电视的信息。
"""
product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""

q_a_pair = f"""
顾客的信息: ```{customer_message}```
产品信息: ```{product_information}```
代理的回复: ```{final_response_to_customer}```

回复是否正确使用了检索的信息？
回复是否充分地回答了问题？

输出 Y 或 N
"""
# प्रासंगिकता निर्धारित करें
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

Y


In [6]:
another_response = "life is like a box of chocolates"
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{another_response}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question?

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages)
print(response)

N


In [ ]:
another_response = "生活就像一盒巧克力"
q_a_pair = f"""
顾客的信息: ```{customer_message}```
产品信息: ```{product_information}```
代理的回复: ```{another_response}```

回复是否正确使用了检索的信息？
回复是否充分地回答了问题？

输出 Y 或 N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages)
print(response)

N


तो आप देख सकते हैं कि मॉडल जेनरेट किए गए आउटपुट की गुणवत्ता के बारे में फीडबैक देने में सक्षम है। आप इस फीडबैक का उपयोग यह तय करने के लिए कर सकते हैं कि उपयोगकर्ता को आउटपुट दिखाना है या नई प्रतिक्रिया उत्पन्न करनी है। आप कई मॉडल प्रतिक्रियाएं उत्पन्न करने का भी प्रयास कर सकते हैं प्रत्येक उपयोगकर्ता क्वेरी के लिए और फिर उपयोगकर्ता को दिखाने के लिए सर्वोत्तम प्रतिक्रिया चुनें, इसलिए आपके पास प्रयास करने के लिए बहुत सारे तरीके हैं।

सामान्य तौर पर, आउटपुट की जांच करने के लिए समीक्षा एपीआई का उपयोग करना एक अच्छा विचार है, हालांकि, मुझे लगता है कि ज्यादातर मामलों में यह अनावश्यक हो सकता हैly जब आप GPT-4 जैसे अधिक उन्नत मॉडल का उपयोग कर रहे हों।

वास्तव में, हमने बहुत से लोगों को वास्तविक उत्पादन परिवेश में यह दृष्टिकोण अपनाते हुए नहीं देखा है, यह सिस्टम में विलंबता और लागत भी जोड़ता है क्योंकि यदि आपके एप्लिकेशन या उत्पाद में केवल त्रुटि दर है तो आपको अतिरिक्त कॉल और अतिरिक्त टोकन के लिए इंतजार करना होगा 0.0000001%, तो शायद आप इस दृष्टिकोण को आज़मा सकते हैं लेकिन सामान्य तौर पर, हम वास्तविक अनुप्रयोगों में इस दृष्टिकोण को अपनाने की अनुशंसा नहीं करते हैं।

अगले अध्याय में, हम वह सब कुछ जोड़ देंगे जो हम चाहते हैंमूल्यांकन इनपुट भाग, प्रसंस्करण भाग और एंड-टू-एंड सिस्टम बनाने के लिए आउटपुट की जाँच में अर्जित किया गया।